In [1]:
import os

# --- 1. SETUP CACHE (Prevents Permission Errors) ---
# Forces Hugging Face to use a local directory you have write access to
os.environ["HF_HOME"] = "./hf_cache"
os.environ["TRANSFORMERS_CACHE"] = "./hf_cache"
os.environ["HF_DATASETS_CACHE"] = "./hf_cache"
os.environ["XDG_CACHE_HOME"] = "./hf_cache"
os.environ["MPLCONFIGDIR"] = "./mpl_cache"

os.makedirs("./hf_cache", exist_ok=True)
os.makedirs("./mpl_cache", exist_ok=True)

print("HF cache set to:", os.environ["HF_HOME"])

import torch
import torch.nn as nn
import torch.nn.utils.prune as prune  # Added for Structured Pruning
from torch.optim import AdamW
from torch.utils.data import DataLoader, Subset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from datasets import load_dataset
from tqdm.auto import tqdm
import numpy as np
import copy
import pandas as pd
import warnings
from sklearn import metrics
import importlib

# --- NEW IMPORTS FOR COMPRESSION ---
import gzip           # For Entropy Coding (File Size Fix)
import pickle         # For saving model artifacts
import scipy.sparse   # For sparse matrix operations

print(f"Torch Version: {torch.__version__}")

# Load Custom Modules
import quantizer
import utils
importlib.reload(utils) 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
warnings.filterwarnings("ignore")

HF cache set to: ./hf_cache


/scratch/dt/adaptivfloat/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch Version: 2.9.0+cu128
Using device: cuda


In [2]:
# --- REPLACEMENT FUNCTIONS FOR SAVING ---

def pack_int4(int8_tensor):
    """
    Packs integer values (-8 to 7) into 4-bit containers.
    CORRECTED ORDER: Index 0 -> High Nibble, Index 1 -> Low Nibble.
    """
    data = int8_tensor.flatten()
    
    # Shift range: -8..7 -> 0..15
    u4 = (data + 8).astype(np.uint8)
    
    # Pad if odd length
    if len(u4) % 2 != 0:
        u4 = np.append(u4, 8) # Pad with 0 (which maps to 8)
        
    # Correct Pairing
    high = u4[0::2] # Index 0, 2, 4...
    low = u4[1::2]  # Index 1, 3, 5...
    
    # Pack: High << 4 | Low
    packed = (high << 4) | low
    return packed

def save_compressed_model(model, profile, filename):
    if not filename.endswith(".gz"): filename += ".gz"
    print(f"📦 Saving with Percentile-Calibrated Per-Row Scaling to {filename}...")
    
    compressed_dict = {
        'meta': {'version': 'v4_calibrated_row_scale'},
        'weights': {}
    }
    
    for name, param in model.named_parameters():
        if 'weight' not in name or param.dim() < 2:
            compressed_dict['weights'][name] = param.detach().cpu().numpy().astype(np.float32)
            continue

        tensor = param.detach()
        precision = profile.get(name, 'INT8')
        
        # 1. Structured Pruning (Identify Active Rows)
        row_magnitudes = torch.sum(torch.abs(tensor), dim=1)
        active_indices = torch.nonzero(row_magnitudes).flatten().cpu().numpy()
        reduced_tensor = tensor[active_indices, :]
        
        # --- THE FIX: Use Numpy for Large Layers (Embeddings) ---
        if precision == 'INT4':
            # 1. Move absolute values to CPU Numpy (Numpy handles >16M elements easily)
            abs_cpu_np = torch.abs(reduced_tensor).detach().cpu().numpy().astype(np.float32)
            
            # 2. Calculate 99.9th percentile using Numpy
            limit_val = np.percentile(abs_cpu_np, 99.9)
            
            # 3. Move the scalar limit back to the original GPU device
            limit = torch.tensor(limit_val, device=tensor.device, dtype=torch.float32)
            
            # Clip the tensor
            reduced_tensor = torch.clamp(reduced_tensor, min=-limit, max=limit)
        # --------------------------------------------------------

        # 2. PER-ROW SCALING
        max_val_per_row, _ = torch.max(torch.abs(reduced_tensor), dim=1, keepdim=True)
        max_val_per_row[max_val_per_row == 0] = 1.0 
        
        target_max = 7.0 if precision == 'INT4' else 127.0
        scales = max_val_per_row / target_max
        
        # 3. Quantize
        quantized_int = torch.round(reduced_tensor / scales)
        
        min_int = -8 if precision == 'INT4' else -128
        max_int = 7 if precision == 'INT4' else 127
        quantized_int = torch.clamp(quantized_int, min_int, max_int)
        
        # 4. Pack
        quantized_np = quantized_int.cpu().numpy().astype(np.int8)
        packed_data = pack_int4(quantized_np) if precision == 'INT4' else quantized_np
        fmt = 'structured_int4_rows' if precision == 'INT4' else 'structured_int8_rows'
            
        # 5. Save
        compressed_dict['weights'][name] = {
            'format': fmt,
            'data': packed_data,
            'row_map': active_indices.astype(np.int32),
            'scale': scales.cpu().numpy().astype(np.float32),
            'original_shape': tensor.shape
        }

    with gzip.open(filename, 'wb') as f:
        pickle.dump(compressed_dict, f)
    
    size_mb = os.path.getsize(filename) / 1024**2
    print(f"   Saved Size: {size_mb:.2f} MB")

# def save_compressed_model(model, profile, filename):
#     if not filename.endswith(".gz"): filename += ".gz"
#     print(f"📦 Saving with High-Precision Per-Row Scaling to {filename}...")
    
#     compressed_dict = {
#         'meta': {'version': 'v3_per_row_scale'},
#         'weights': {}
#     }
    
#     for name, param in model.named_parameters():
#         if 'weight' not in name or param.dim() < 2:
#             compressed_dict['weights'][name] = param.detach().cpu().numpy().astype(np.float32)
#             continue

#         tensor = param.detach()
#         precision = profile.get(name, 'INT8')
        
#         # 1. Structured Pruning
#         row_magnitudes = torch.sum(torch.abs(tensor), dim=1)
#         active_indices = torch.nonzero(row_magnitudes).flatten().cpu().numpy()
#         reduced_tensor = tensor[active_indices, :]
        
#         # 2. PER-ROW SCALING (The Upgrade)
#         # Instead of max(whole_tensor), we calculate max(each_row)
#         # Shape: [Rows, 1]
#         max_val_per_row, _ = torch.max(torch.abs(reduced_tensor), dim=1, keepdim=True)
        
#         # Avoid division by zero
#         max_val_per_row[max_val_per_row == 0] = 1.0
        
#         target_max = 7.0 if precision == 'INT4' else 127.0
#         scales = max_val_per_row / target_max
        
#         # 3. Quantize
#         # Broadcasting handles the division automatically: [Rows, Cols] / [Rows, 1]
#         quantized_int = torch.round(reduced_tensor / scales)
        
#         min_int = -8 if precision == 'INT4' else -128
#         max_int = 7 if precision == 'INT4' else 127
#         quantized_int = torch.clamp(quantized_int, min_int, max_int)
        
#         # 4. Pack
#         quantized_np = quantized_int.cpu().numpy().astype(np.int8)
#         packed_data = pack_int4(quantized_np) if precision == 'INT4' else quantized_np
#         fmt = 'structured_int4_rows' if precision == 'INT4' else 'structured_int8_rows'
            
#         # 5. Save
#         compressed_dict['weights'][name] = {
#             'format': fmt,
#             'data': packed_data,
#             'row_map': active_indices.astype(np.int32),
#             'scale': scales.cpu().numpy().astype(np.float32), # Save as VECTOR now
#             'original_shape': tensor.shape
#         }

#     with gzip.open(filename, 'wb') as f:
#         pickle.dump(compressed_dict, f)
    
#     size_mb = os.path.getsize(filename) / 1024**2
#     print(f"   Saved Size: {size_mb:.2f} MB")

In [3]:
# --- 2. CONFIGURATION CLASS ---
class ExperimentConfig:
    def __init__(self, model_name="bert-base-uncased", task="sst2"):
        self.model_name = model_name
        self.task = task
        self.batch_size = 16 
        self.lr = 1e-5
        
        # --- CRITICAL SETTINGS FOR STRUCTURED PRUNING ---
        self.pruning_amount = 0.15  # Remove 30% of rows/neurons
        
        self.epochs_finetune = 10   # Baseline training
        self.epochs_recovery = 10   # INCREASED to 3 to heal "Broken" rows
    
        self.sensitivity_threshold = 0.015 
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Define labels based on task
        if task in ['sst2', 'qnli', 'rte', 'mrpc', 'qqp']:
            self.num_labels = 2
        elif task in ['mnli']:
            self.num_labels = 3
        elif task in ['stsb']:
            self.num_labels = 1
        else:
            self.num_labels = 2 

# Define Quantizers globally
quant_af4_func = quantizer.quant_af4_func
quant_af8_func = quantizer.quant_af8_func

print("✅ Config Setup Complete.")

# --- 3. HELPER FUNCTIONS ---

def get_single_sample(loader, device):
    batch = next(iter(loader))
    if isinstance(batch, dict):
        return {k: v[0:1].to(device) for k, v in batch.items() if k != 'label'}
    else:
        # Some loaders return list/tuples
        if hasattr(batch, 'items'): # Check if it's a dict-like object
             return {k: v[0:1].to(device) for k, v in batch.items() if k != 'label'}
        return batch[0][0:1].to(device)

def prepare_data(cfg):
    print(f"--- [Data] Loading {cfg.task} for {cfg.model_name} ---")
    
    # Use local cache to prevent download errors
    tokenizer = AutoTokenizer.from_pretrained(cfg.model_name, cache_dir="./hf_cache")
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Map GLUE tasks to their specific input column names
    task_to_keys = {
        "sst2": ("sentence", None),
        "mnli": ("premise", "hypothesis"),
        "qnli": ("question", "sentence"),
        "mrpc": ("sentence1", "sentence2"),
        "rte":  ("sentence1", "sentence2"),
        "qqp":  ("question1", "question2"),
    }
    
    key1, key2 = task_to_keys.get(cfg.task, ("sentence", None))
        
    def tokenize_fn(batch):
        if key2 is None:
            return tokenizer(batch[key1], padding='max_length', truncation=True, max_length=128)
        else:
            return tokenizer(batch[key1], batch[key2], padding='max_length', truncation=True, max_length=128)

    # Load from local cache
    raw_dataset = load_dataset("glue", cfg.task, cache_dir="./hf_cache")
    tokenized_dataset = raw_dataset.map(tokenize_fn, batched=True)
    
    # Set format for PyTorch
    tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    train_loader = DataLoader(tokenized_dataset['train'], batch_size=cfg.batch_size, shuffle=True)
    
    # MNLI validation split handling
    if cfg.task == 'mnli':
        val_split = 'validation_matched'
    else:
        val_split = 'validation'
        
    val_loader = DataLoader(tokenized_dataset[val_split], batch_size=cfg.batch_size)
    
    # Quick Loader (Use first 500 samples for speed)
    val_dataset = val_loader.dataset
    if len(val_dataset) > 500:
        subset_indices = list(range(500))
    else:
        subset_indices = list(range(len(val_dataset)))
        
    quick_loader = DataLoader(Subset(val_dataset, subset_indices), batch_size=cfg.batch_size)
    
    return train_loader, val_loader, quick_loader

✅ Config Setup Complete.


In [4]:
# --- 4. CORE PIPELINE FUNCTIONS ---

def train_fp32_baseline(cfg, train_loader, val_loader):
    print(f"--- [Phase 0] Fine-Tuning Baseline: {cfg.model_name} ---")
    
    # Use cache_dir to prevent permission errors
    model = AutoModelForSequenceClassification.from_pretrained(
        cfg.model_name, 
        num_labels=cfg.num_labels,
        cache_dir="./hf_cache"
    ).to(cfg.device)
    
    if model.config.pad_token_id is None:
        model.config.pad_token_id = model.config.eos_token_id
        
    optimizer = AdamW(model.parameters(), lr=cfg.lr)
    scheduler = get_linear_schedule_with_warmup(optimizer, 0, len(train_loader) * cfg.epochs_finetune)
    
    model.train()
    
    for epoch in tqdm(range(cfg.epochs_finetune), desc="Training Epochs"):
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}", leave=False):
            optimizer.zero_grad()
            batch = {k: v.to(cfg.device) for k, v in batch.items()}

            if 'label' in batch:
                batch['labels'] = batch.pop('label')
                
            # HF models handle labels automatically in forward()
            outputs = model(**batch)
            outputs.loss.backward()
            optimizer.step()
            scheduler.step()
            
    model.eval()
    
    print("Evaluating FP32 Baseline...")
    fp32_results = {}
    
    # Use simple accuracy check
    acc = utils.get_accuracy(model, val_loader, cfg.device)
    fp32_results['accuracy'] = acc
    fp32_results['size_mb'] = utils.get_model_size_mb(model)
    
    # Rough Energy Estimate
    flops = 1e9 # Placeholder
    fp32_results['Energy_J'] = flops * 3.7e-9 
    
    print(f"Baseline Acc: {fp32_results['accuracy']:.4f}")
    return model, fp32_results

def prune_and_recover(model, train_loader, cfg):
    print(f"--- [Phase 1] Structured Pruning ({cfg.pruning_amount*100}%) & Recovery ---")
    
    model_pruned = copy.deepcopy(model)
    
    # 1. Apply STRUCTURED Pruning (Removes entire rows)
    utils.apply_structured_pruning(model_pruned, cfg.pruning_amount)
    
    # 2. Recovery Training (Crucial for Structured)
    model_pruned.train()
    optimizer = AdamW(model_pruned.parameters(), lr=cfg.lr) 
    scheduler = get_linear_schedule_with_warmup(optimizer, 0, len(train_loader) * cfg.epochs_recovery)
    
    for epoch in tqdm(range(cfg.epochs_recovery), desc="Recovery Epochs"):
        for batch in tqdm(train_loader, desc=f"Recovery {epoch+1}", leave=False):
            optimizer.zero_grad()
            batch = {k: v.to(cfg.device) for k, v in batch.items()}

            if 'label' in batch:
                batch['labels'] = batch.pop('label')

            outputs = model_pruned(**batch)
            outputs.loss.backward()
            optimizer.step()
            scheduler.step()
    
    # 3. Make Pruning Permanent (Fixes the zeros)
    utils.make_pruning_permanent(model_pruned)
    
    model_pruned.eval()
    return model_pruned

def run_greedy_search(model_pruned_base, quick_loader, cfg):
    print("--- [Phase 2] Sensitivity-Aware Mixed-Precision Search ---")
    
    # Get all weight matrices (Linear layers)
    layer_names = [name for name, param in model_pruned_base.named_parameters() 
                   if 'weight' in name and param.dim() > 1]
    
    # 1. Baseline INT8 (All layers quantized to 8-bit)
    model_int8 = copy.deepcopy(model_pruned_base).to(cfg.device)
    # We use the updated utils function which handles profile application
    utils.apply_quantization_to_model(model_int8, {'default': 'INT8'}) # Apply flat INT8
    baseline_acc = utils.get_accuracy(model_int8, quick_loader, cfg.device)
    print(f"   Baseline (INT8): {baseline_acc * 100:.2f}%")
    del model_int8

    # 2. SENSITIVITY ANALYSIS (The "S" in SAMPQ)
    # We test each layer: What happens if ONLY this layer is INT4?
    print("   Calculating Layer Sensitivity...")
    sensitivity_scores = []
    
    for layer_name in tqdm(layer_names, desc="Sensitivity Analysis", leave=False):
        # Create a profile where everything is INT8...
        temp_profile = {n: 'INT8' for n in layer_names}
        # ...except this specific layer is INT4
        temp_profile[layer_name] = 'INT4'
        
        test_model = copy.deepcopy(model_pruned_base).to(cfg.device)
        utils.apply_quantization_to_model(test_model, temp_profile)
        acc = utils.get_accuracy(test_model, quick_loader, cfg.device)
        del test_model
        
        # Drop = Baseline - New_Acc (Higher drop = More sensitive)
        drop = baseline_acc - acc
        sensitivity_scores.append((layer_name, drop))

    # 3. SORT LAYERS (Least Sensitive First)
    # We want to quantize the "safe" layers first
    sensitivity_scores.sort(key=lambda x: x[1]) 
    sorted_layers = [x[0] for x in sensitivity_scores]
    
    print(f"   Most Robust Layer: {sorted_layers[0]} (Drop: {sensitivity_scores[0][1]:.4f})")
    print(f"   Most Sensitive Layer: {sorted_layers[-1]} (Drop: {sensitivity_scores[-1][1]:.4f})")

    # 4. GREEDY OPTIMIZATION LOOP
    final_profile = {layer: 'INT8' for layer in layer_names}
    
    # We allow accuracy to drop by 'sensitivity_threshold' total
    current_acc_limit = baseline_acc - cfg.sensitivity_threshold
    
    pbar = tqdm(sorted_layers, desc="Greedy Optimization")
    
    for layer_name in pbar:
        # Attempt to demote this layer to INT4
        final_profile[layer_name] = 'INT4'
        
        # Test the cumulative effect
        test_model = copy.deepcopy(model_pruned_base).to(cfg.device)
        utils.apply_quantization_to_model(test_model, final_profile)
        acc = utils.get_accuracy(test_model, quick_loader, cfg.device)
        del test_model
        
        # If accuracy drops too much below our limit, REVERT it
        if acc < current_acc_limit:
            final_profile[layer_name] = 'INT8' # Revert
        else:
            # Keep INT4
            pass
            
        pbar.set_postfix({'Acc': f"{acc:.4f}", 'INT4_Count': list(final_profile.values()).count('INT4')})

    return final_profile

def build_and_evaluate_final(model_pruned, profile, val_loader, cfg):
    print("--- [Phase 3] Building & Evaluating Final Hybrid Model ---")
    
    # We work on the pruned model (which is already permanent)
    model_hybrid = copy.deepcopy(model_pruned)
    
    # Apply Quantization Profile (Fake Quant for eval)
    utils.apply_quantization_to_model(model_hybrid, profile)
    
    # Evaluate
    acc = utils.get_accuracy(model_hybrid, val_loader, cfg.device)
    
    # Metrics
    results = {}
    results['accuracy'] = acc
    results['sparsity'] = cfg.pruning_amount * 100
    
    int4_c = list(profile.values()).count('INT4')
    int8_c = list(profile.values()).count('INT8')
    avg_bits = ((int4_c * 4) + (int8_c * 8)) / (int4_c + int8_c) if (int4_c + int8_c) > 0 else 8.0
    
    # Theoretical Size Calculation
    # Structured pruning reduces parameter count, so we use current numel()
    total_params = sum(p.numel() for p in model_hybrid.parameters())
    hybrid_size = (total_params * avg_bits) / (8 * 1024**2)
    
    return {
        "Model": cfg.model_name,
        "Hybrid_Acc": results['accuracy'],
        "Sparsity": results['sparsity'],
        "Avg_Bits": avg_bits,
        "Hybrid_Size_MB": hybrid_size, # Theoretical
        "Task": cfg.task
    }

In [ ]:
# Create output folder
folder_name = "compressed_models_structured"
os.makedirs(folder_name, exist_ok=True)

# --- 5. MAIN EXECUTION LOOP ---

models_to_test = [
    # "prajjwal1/bert-tiny",                  
    # "google/electra-small-discriminator",   
    # "distilbert-base-uncased",              
    # "bert-base-uncased",                    
    # "roberta-base",                         
    "gpt2",                                 
    "albert-base-v2",                       
]

# CHANGE THIS TO "qnli", "mnli", "mrpc"
target_task = "mnli"  

paper_results = []

print(f"🚀 STARTING STRUCTURED RUN: {len(models_to_test)} Models on {target_task}")

for m_name in models_to_test:
    print(f"\n{'='*40}\nRunning Pipeline for: {m_name} on {target_task}\n{'='*40}")
    
    cfg = ExperimentConfig(model_name=m_name, task=target_task)
    
    try:
        # Prepare Data
        train_loader, val_loader, quick_loader = prepare_data(cfg)
        
        # Phase 0: Baseline
        model_fp32, fp32_res = train_fp32_baseline(cfg, train_loader, val_loader)
        
        # Phase 1: Structured Pruning (Removes Rows)
        model_pruned = prune_and_recover(model_fp32, train_loader, cfg)
        del model_fp32 
        torch.cuda.empty_cache()
        
        # Phase 2: Search (Finds Mixed Precision)
        profile = run_greedy_search(model_pruned, quick_loader, cfg)
        
        # Phase 3: Final Eval
        final_stats = build_and_evaluate_final(model_pruned, profile, val_loader, cfg)
        
        # --- PHASE 4: SAVE REAL COMPRESSED MODEL ---
        
        # 1. Prepare model for saving
        # Note: model_pruned already has rows removed permanently.
        # We assume the saver handles the quantization packing based on 'profile'.
        model_to_save = copy.deepcopy(model_pruned)
        
        # 2. Save it
        safe_name = m_name.replace("/", "_")
        filename = f"{folder_name}/{safe_name}_{target_task}.pkl" 
        
        # This calls the NEW Structured Saver (from Cell 3)
        save_compressed_model(model_to_save, profile, filename)
        
        del model_to_save 
        # -------------------------------------------------

        # Metrics
        final_stats['FP32_Acc'] = fp32_res['accuracy']
        final_stats['FP32_Energy_J'] = fp32_res['Energy_J']
        final_stats['Acc_Loss'] = fp32_res['accuracy'] - final_stats['Hybrid_Acc']
        final_stats['Task'] = target_task
        
        paper_results.append(final_stats)
        
        # Save Checkpoint CSV
        pd.DataFrame(paper_results).to_csv(f"paper_results_{target_task}_checkpoint.csv")
        
    except Exception as e:
        print(f"⚠️ Failed on {m_name}: {e}")
        import traceback
        traceback.print_exc()

print("\n\n📊 FINAL RESULTS 📊")
df = pd.DataFrame(paper_results)
pd.options.display.float_format = '{:.4f}'.format
print(df)
df.to_csv(f"research_paper_final_results_{target_task}.csv")

🚀 STARTING STRUCTURED RUN: 2 Models on mnli

Running Pipeline for: gpt2 on mnli
--- [Data] Loading mnli for gpt2 ---


Map: 100%|██████████| 9847/9847 [00:00<00:00, 16383.72 examples/s]


--- [Phase 0] Fine-Tuning Baseline: gpt2 ---


Loading weights: 100%|██████████| 148/148 [00:00<00:00, 1718.55it/s, Materializing param=transformer.wte.weight]             
GPT2ForSequenceClassification LOAD REPORT from: gpt2
Key                  | Status     | 
---------------------+------------+-
h.{0...11}.attn.bias | UNEXPECTED | 
score.weight         | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.
Training Epochs:  20%|██        | 2/10 [57:26<3:49:42, 1722.84s/it]